# Exploring the bos

![overview of the bos](system_diagram.png "Title")

The System Model (sysmod) is the knowledge graph used to represent a building and its points.

System models are conventionally built up using Brick Schema, ASHRAE 223P, and Real Estate Core, though use can you any RDF schema you want (e.g., Haystack 4 is fine.)

The `sysmod` service runs by default on port 2821 (0xB05) of your local machine.

To get started import the `bospy` package.

If you don't have these package  run:
``` shell 
pip install grpcio grpcio-tools
pip install bospy
```

If you are not running the bos on your local machine you'll need to set the environment variables `SYSMOD_ADDR` and `DEVCTRL_ADDR`.

In [4]:
import os
os.environ['SYSMOD_ADDR'] = "nuc:2821"
os.environ['DEVCTRL_ADDR'] = "nuc:2822"
os.environ['HIS_ADDR'] = "nuc:2823"

In [5]:
import bospy.bos as bos
print(bos.VERSION)

0.0.4


## Getting all Points

To see all the points in the sysmod we can call `QueryPoints()` with no arguments.

In [6]:
all_pts = bos.QueryPoints()
print("There are {} points in the sysmod:".format(len(all_pts)))
print(all_pts)


There are 12 points in the sysmod:
['bos://localhost/dev/1/pts/3', 'bos://localhost/dev/1/pts/4', 'bos://localhost/dev/1/pts/5', 'bos://localhost/dev/1/pts/6', 'bos://localhost/dev/2/pts/1', 'bos://localhost/dev/3/pts/2', 'bos://localhost/dev/3/pts/3', 'bos://localhost/dev/1/pts/1', 'bos://localhost/dev/1/pts/2', 'bos://localhost/dev/2/pts/2', 'bos://localhost/dev/2/pts/3', 'bos://localhost/dev/3/pts/1']


In the bos's internal namespace points are tracked by unique URI. These URIs are not meant to be interacted with directly. Instead it's assumed they'll be passed directly to other functions. 

For example, maybe we want to the *names* of all our points. We can view them like this:

In [7]:
print("There are {} points in the sysmod. Their names are:".format(len(all_pts)))
for p in all_pts:
    name = bos.PointToName(p)
    print("   ", name)

There are 12 points in the sysmod. Their names are:
    air_temp_setpoint
    humidity
    status
    power_draw
    air_temp_west
    air_temp_stpt_east
    humidity_east
    co2
    air_temp
    air_temp_stpt_west
    humidity_west
    air_temp_east


People familiar with building automation systems (BASs) are likely used to accessing points by name. We can get a point's uri by name with `NameToPoint()`.

*Programs written by referencing points by name are not portable. If you're interested in developing portable applications consider using parametric point selections (e.g,. a combination of `type` and `location` params). Full SPARQL queries will also be supported soon*

In [8]:
name = 'air_temp_setpoint'
pt = bos.NameToPoint(name)
print('the point named {} is accessed via {}'.format(name, pt))

the point named air_temp_setpoint is accessed via bos://localhost/dev/1/pts/3


## Getting and Setting Values

If you're exploring the sysmod chances are you are looking for some points to read or write. We can read a point with `Get()` and write it with `Set()`.

`Get()` and `Set()` make use of a service called `devctrl` (device control).

By default `devctrl` is runs at `localhost:2822`.

In [9]:
name = 'air_temp_setpoint'
pt = bos.NameToPoint(name)
values = bos.Get(pt)
print(name, "=", values[pt])


air_temp_setpoint = 22.5


`Get()` also accepts list of points. It returns a map of point uris and their current values. As we've seen we can swap those uris for names with `PointToName()`.

In [10]:
values = bos.Get(all_pts[:3])
for p, v in values.items():
    name = bos.PointToName(p)
    print(name, "=", v)

air_temp_setpoint = 22.25
humidity = 55
status = True


We can set values with `Set()` by passing one point and one value like this.

In [12]:
# check the value of 'air_temp_setpoint'
name = 'air_temp_setpoint'
pt = bos.NameToPoint(name)
values = bos.Get(pt)
print("t=0",name, "=", values[pt])

# change the setpoint
new_value = 18.5
ok = bos.Set(pt, new_value)
if ok:
    print("t=1", name, "<-", new_value, "(success)")
else:
    print("t=1", name, "not set (error)")

# confirm it was set correctly
values = bos.Get(pt)
print("t=2", name, "=", values[pt])

t=0 air_temp_setpoint = 23.0
t=1 air_temp_setpoint <- 18.5 (success)
t=2 air_temp_setpoint = 18.5


## Getting Fancy

### Average zoom temp
Let's say the location 'lab' has *n* space temperature sensors and we want to decide if the average is acceptable. We can do something like this:

In [13]:
loc = 'lab'
pts = bos.QueryPoints(types='brick:Air_Temperature_Sensor', locations=loc)
values = bos.Get(pts)

import numpy as np
avg = np.array([*values.values()]).mean()
print("The average space temperature at '{}' is: {:.1f}°C (based on {} sensors)".format(loc, avg, len(pts)))

The average space temperature at 'lab' is: 20.2°C (based on 3 sensors)


If the average is too high we can lower the set point of *all* setpoints in 'lab' with.

In [14]:
stpts = bos.QueryPoints(types='brick:Air_Temperature_Setpoint', locations=loc)
ok = bos.Set(stpts, 18)

# check our work
values = bos.Get(stpts)
for k, v in values.items():
    name = bos.PointToName(k)
    print("{} = {}°C".format(name, v))

air_temp_stpt_west = 18.0°C
air_temp_stpt_east = 18.0°C
air_temp_setpoint = 18.0°C
